In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import nltk

In [ ]:
train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")

In [ ]:
for index,row in train.sample(5).iterrows():
    print(str(row['id']) + ":" + row['text'] + " =" + str(row['target']))
    print (" ")

In [ ]:
train.isnull().sum()


In [ ]:
train.shape

Tasks:
    
- get word counts for 0 and 1 labels
https://towardsdatascience.com/exploratory-data-analysis-for-natural-language-processing-ff0046ab3571 
https://monkeylearn.com/text-analysis/ 

In [ ]:
train.head()

In [ ]:
import collections
from nltk.corpus import stopwords  
import seaborn as sns


Refs: 

https://www.geeksforgeeks.org/removing-stop-words-nltk-python/ --> for stopwords

In [ ]:
# get list of stopwords
nltk.download("stopwords")
stop_words = set(stopwords.words('english'))

In [ ]:
# Get a list of all text entries
tweet_collection = [tweet for tweet in train['text']]

# concatenate into 1 large text base 
corpus = " ".join(tweet_collection)

Let's start by doing a word frequency analysis to see what the most popular words are

In [ ]:
counter = collections.Counter(corpus.lower().split())

In [ ]:
#establish stopped word dictionary
stopped_words = collections.defaultdict(int)

In [ ]:
# Searching and removing stopwords
for stop_word in list(stop_words):
    if stop_word in counter.keys():
        stopped_words[stop_word] = counter[stop_word]
        del counter[stop_word]
        

In [ ]:
stopped_words

In [ ]:
pop_words = counter.most_common()

In [ ]:
x = []
y = []
for word, count  in pop_words[:10]:
    x.append(word)
    y.append(count)

In [ ]:
sns.barplot(x = y, y = x)

Have a good skeleton, but need to remove punctuation and weird characters.

In [ ]:
import statistics

In [ ]:
# Using tweet collection from above exploration
tweet_collection[0]

In [ ]:
# slightly complicated code, and perhaps a little inefficient?
# basically split each sentence into a list of words (while still maintaining the initial list)
# then get word lengths and average these numbers (thus collapsing the internal lists)
avg_word_length = list(map(lambda x: statistics.mean(len(y) for y in x), [x.split() for x in tweet_collection]))

In [ ]:
# list(map(lambda x: len(x.split()), tweet_collection)) --> equivalent code for below

In [ ]:
word_counts = [len(x.split()) for x in tweet_collection]

In [ ]:
sns.scatterplot(x = word_counts, y = avg_word_length)

There does seem to be a slightly decreasing trend in tweet lengths and the avg length of the word (mostly explained by variability d/t sentence length) --> not very useful

Sentiment Analysis Segment
https://medium.com/analytics-vidhya/simplifying-social-media-sentiment-analysis-using-vader-in-python-f9e6ec6fc52f 

Hutto, C.J. & Gilbert, E.E. (2014). VADER: A Parsimonious Rule-based Model for Sentiment Analysis of Social Media Text. Eighth International Conference on Weblogs and Social Media (ICWSM-14). Ann Arbor, MI, June 2014.

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()



In [ ]:
pos_test = "The dog is happy"
neg_test = "You look so ugly"

In [ ]:
sid.polarity_scores(pos_test)

In [ ]:
sid.polarity_scores(neg_test)